In [0]:
DEBUG = False
FIELD = None
# M31_SECTOR = 'm31_W0'
M31_SECTOR = None
CONFIG_FILES = [
    # '/home/dobos/project/ga_targeting/configs/netflow/SSP/MW/outerdisk_common.py',
    # '/home/dobos/project/ga_targeting/configs/netflow/SSP/MW/outerdisk_l90_b28.py' 
    # '/home/dobos/project/ga_targeting/configs/netflow/SSP/MW/outerdisk_l90_bm28.py' 
    # '/home/dobos/project/ga_targeting/configs/netflow/SSP/MW/outerdisk_l90_b25.py' 
    
    # '/home/dobos/project/ga_targeting/configs/netflow/SSP/MW/outerdisk_l180_b16.py',
    #'/home/dobos/project/ga_targeting/configs/netflow/SSP/MW/outerdisk_l180_b29.py',
    '/home/mihoishigaki/Subaru-PFS-GA/ga_targeting/configs/netflow/SSP/MW/outerhalo_l181_bm60.py',
    # '/home/dobos/project/ga_targeting/configs/netflow/SSP/CC/crosscalib_ra288_dec-11.py'
    # '/home/dobos/project/ga_targeting/configs/netflow/SSP/CC/crosscalib_ra288_dec-22.py'
    # '/home/dobos/project/ga_targeting/configs/netflow/SSP/m31/m31_W0.py'
]
# CENTER = (273.525, 60.88)
# CENTER, PA = (340.4, 26.50), 120       # outerdisk_l90_bm28_faint
# CENTER = (290.0, -22.0)
# CENTER = (280.0, 60.5)                   # outerdisk_l90_b25
# CENTER = (10.6847, 41.2687)                   # m31
CENTER = (38.202654, -9.0460723)                   # outer disk_l180_b16
PA = 0                                   # Can be anything, not used in this script

# Observation time frame in HST
TIME_FROM = '2026-01-26 18:00:00'
TIME_TO = '2026-01-26 23:00:00'
TIME_MID = '2026-01-26 18:00:00'

In [0]:
import os, sys
import dateutil
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d

In [0]:
plt.rc('font', size=6)

In [0]:
%load_ext autoreload
%autoreload 2

In [0]:
if DEBUG and 'debug' not in globals():
    import debugpy
    debugpy.listen(('0.0.0.0', 5698))
    debug = True

In [0]:
from astropy.time import Time, TimeDelta
from astropy.coordinates import SkyCoord, EarthLocation, AltAz, get_body
from astroplan import Observer
from astroplan.moon import moon_phase_angle, moon_illumination
import astropy.units as u

from pfs.utils.coordinates import Subaru_POPT2_PFS

from pfs.ga.targeting.targets.dsph import GALAXIES as DSPH_FIELDS
from pfs.ga.targeting.targets.m31 import M31_SECTORS
from pfs.ga.targeting.instrument import *
from pfs.ga.common.diagram import CMD, CCD, FOV, FP, ColorAxis, MagnitudeAxis
from pfs.ga.common.photometry import Photometry, Magnitude, Color
from pfs.ga.targeting.projection import Pointing, WcsProjection
from pfs.ga.targeting.config.netflow import NetflowConfig

In [0]:
if FIELD is not None:
    field = DSPH_FIELDS[FIELD]
    center = field.get_center()
elif M31_SECTOR is not None:
    sector = M31_SECTORS[M31_SECTOR]
    center = sector.get_center()
else:
    center = Pointing(*CENTER, posang=PA)

center

# Load the config files

In [0]:
if FIELD is not None:
    config = field.get_netflow_config()
elif M31_SECTOR is not None:
    config = sector.get_netflow_config()
else:
    config = NetflowConfig.default()
    
config.load(CONFIG_FILES, ignore_collisions=True)

In [0]:
config.pointings

In [0]:
config.instrument_options

# Plot the pointings

In [0]:
wcs = WcsProjection(center, proj='TAN')
fov = FOV(projection=wcs)

In [0]:
f, ax = plt.subplots(1, 1, figsize=(8, 8), dpi=240, subplot_kw=dict(projection=wcs.wcs))

# fov.plot_observation(ax, obs, c='lightgrey', size=0.5)

# Plot the pointing center
# fov.plot(ax, [field.pos.ra.value], [field.pos.dec.value], fmt='+r', ms=10, native_frame='world')

for pp in config.pointings:
    pointing = pp.get_pointing()
    if pointing.obs_time is None:
        pointing.obs_time = TIME_MID
    wfc = SubaruWFC(pointing)
    pfi = SubaruPFI(wfc, instrument_options=config.instrument_options)

    pfi.plot_focal_plane(ax, fov, corners=True, c='r', ls='-')
    pfi.plot_focal_plane(ax, fov, fill=True, alpha=0.2)

ax.set_xlim(1, -3)
ax.set_ylim(-3, 1)
ax.grid()

# Calculate and plot the elevation and rotator angle

In [0]:
obs = Observer.at_site("Subaru", timezone="US/Hawaii")
hawaii_tz = TimeDelta(-10 * u.hr)

In [0]:
config.pointings

In [0]:
time_from = Time(dateutil.parser.parse(TIME_FROM)) - hawaii_tz
time_to = Time(dateutil.parser.parse(TIME_TO)) - hawaii_tz

time_from, time_to

In [0]:
date_from = np.floor(time_from.jd - 0.5)
date_to = np.ceil(time_to.jd + 0.5)

dates = Time(np.linspace(date_from, date_to + 1, int(date_to - date_from + 2)), format='jd')
dates.to_datetime()

In [0]:
sunrise = obs.sun_rise_time(dates, which='previous') + hawaii_tz
sunset = obs.sun_set_time(dates, which='next') + hawaii_tz

# HST
sunrise.to_datetime(), sunset.to_datetime()

In [0]:
def radecpa_to_altazinr(pos, pa, obs_time):
    altaz = pos.transform_to(AltAz(obstime=obs_time, location=Subaru_POPT2_PFS.Lsbr))
    az = altaz.az.deg
    alt = altaz.alt.deg

    # Instrument rotator angle
    subaru = Subaru_POPT2_PFS.Subaru()
    paa = subaru.radec2inr(pos.ra, pos.dec, obs_time)
    inr = (paa + pa + 180) % 360. - 180.
    # inr = paa + pa

    return alt, az, inr

In [0]:
obs_time = Time(np.linspace(date_from, date_to, 300), format='jd')
obs_time[0], obs_time[-1]

In [0]:
obs_time[0], obs_time[-1]

In [0]:
f, ax = plt.subplots(1, 1, figsize=(7, 5), dpi=240)
ax2 = ax.twinx()

# matplotlib like python datetime
xx = (obs_time + hawaii_tz).to_datetime()

# Shade areas in black when the Sun is below the horizon
for t1, t2 in zip(sunset[:-2].to_datetime(), sunrise[2:].to_datetime()):
    ax.axvspan(t1, t2, color='black', alpha=1)

# Plot the rotator angle for each pointing
for p in config.pointings:
    p = p.get_pointing()
    alt, az, inr = radecpa_to_altazinr(p.pos, p.posang, obs_time=(obs_time))
    # alt, az, inr = radecpa_to_altazinr(p.pos, 0, obs_time=(obs_time))

    inr_mask = np.abs(inr) > 178
    inr[inr_mask] = np.nan

    # Instrument rotator angle
    ax2.plot(xx, inr, c='orange', label='INR')

    # Plot the azimuth angle
    # target_ha = obs.target_hour_angle(obs_time, target=p.pos)
    ax2.plot(xx, ((az + 180.) % 360) - 180., c='green', label='AZ')

# Plot the elevation of the Moon
moon = get_body('Moon', obs_time, Subaru_POPT2_PFS.Lsbr)
moon_phase = moon_illumination(obs_time)
altaz = moon.transform_to(AltAz(obstime=obs_time, location=Subaru_POPT2_PFS.Lsbr))
ax.plot(xx, altaz.alt.degree, '--y', label='MOON')

# Plot target elevation
alt, az, inr = radecpa_to_altazinr(center.pos, 0, obs_time=(obs_time))
ax.plot(xx, alt, '-c', label='ALT')

ax.set_xlim((time_from + hawaii_tz).to_datetime(), (time_to + hawaii_tz).to_datetime())

ax.set_yticks(np.linspace(0, 90, 7))
ax.set_ylim(0, 90)

ax.tick_params(axis='x', labelrotation=90)

ax.set_xlabel('time [HST]')
ax.set_ylabel('elevation')
ax.set_title(f'Field: {FIELD}, moon phase: {moon_phase.mean():0.2f}')

ax.grid(True)

ax2.set_yticks(np.linspace(-180, 180, 7))
ax2.set_ylim(-180, 180)

ax2.set_ylabel('rotator angle')

ax.legend()
ax2.legend()